In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
readdata = pd.read_csv("./May 30/clinical_data.csv").drop(columns=['case_id'])


readdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ajcc_pathologic_stage   991 non-null    int64  
 1   days_to_last_follow_up  991 non-null    float64
 2   primary_diagnosis       991 non-null    object 
 3   prior_malignancy        991 non-null    object 
 4   year_of_diagnosis       991 non-null    float64
 5   prior_treatment         991 non-null    object 
 6   pharm_treatment         991 non-null    int64  
 7   radiation               991 non-null    int64  
 8   ethnicity               991 non-null    object 
 9   race                    991 non-null    object 
 10  vital_status            991 non-null    object 
 11  age_at_index            991 non-null    int64  
 12  year_of_birth           991 non-null    float64
dtypes: float64(3), int64(4), object(6)
memory usage: 100.8+ KB


In [3]:
readdata

,ajcc_pathologic_stage,days_to_last_follow_up,primary_diagnosis,prior_malignancy,year_of_diagnosis,prior_treatment,pharm_treatment,radiation,ethnicity,race,vital_status,age_at_index,year_of_birth
0,1,337.0,"Infiltrating duct carcinoma, NOS",no,2010.0,No,1,0,not hispanic or latino,white,Alive,60,1950.0
1,3,5.0,"Infiltrating duct carcinoma, NOS",no,2010.0,No,0,0,not hispanic or latino,white,Alive,56,1954.0
2,1,759.0,"Lobular carcinoma, NOS",yes,2012.0,No,1,0,not hispanic or latino,white,Alive,61,1951.0
3,5,954.0,"Lobular carcinoma, NOS",yes,2010.0,No,1,1,not hispanic or latino,white,Alive,71,1939.0
4,8,304.0,"Infiltrating duct carcinoma, NOS",no,2009.0,No,1,0,not reported,not reported,Alive,76,1933.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,1,2813.0,"Infiltrating duct carcinoma, NOS",no,2003.0,No,1,1,not hispanic or latino,white,Alive,60,1943.0
987,8,5.0,"Infiltrating duct carcinoma, NOS",no,2013.0,No,0,0,not hispanic or latino,black or african american,Alive,35,1978.0
988,1,1532.0,"Infiltrating duct carcinoma, NOS",no,2009.0,No,1,0,not hispanic or latino,black or african american,Alive,46,1963.0
989,3,375.0,"Lobular carcinoma, NOS",no,2011.0,No,1,1,not hispanic or latino,white,Alive,71,1940.0


### Exploratory Data Visualization

In [4]:
target_variable = 'vital_status'
# Function to visualize categorical features
def visualize_categorical_features(df, target_variable):
    categorical_columns = df.select_dtypes(include=['object']).columns
    categorical_columns = [col for col in categorical_columns if col != target_variable]
    for column in categorical_columns:
        plt.figure(figsize=(6, 5))
        sns.countplot(x=column, hue=target_variable, data=df)
        plt.title(f'Relationship between {column} and {target_variable}')
        plt.xticks(rotation=45)
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f"./plots/{column}.png", bbox_inches='tight')
        plt.clf()

# Visualize categorical features
visualize_categorical_features(readdata, target_variable)

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

In [5]:
# Function to visualize continuous features
def visualize_continuous_features(df, target_variable):
    continuous_columns = df.select_dtypes(include=['float64', 'int64']).columns
    for column in continuous_columns:
        plt.figure(figsize=(10, 5))
        sns.boxplot(x=target_variable, y=column, data=df)
        plt.title(f'Relationship between {column} and {target_variable}')
        plt.xticks(rotation=45)
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f"./plots/{column}.png", bbox_inches='tight')
        plt.clf()


# Visualize continuous features
visualize_continuous_features(readdata, target_variable)

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

#### Data Analysis

In [6]:
# separate features from target variable and encode y as 0 or 1

df = readdata.copy().drop(columns='vital_status', axis=1)
y = np.array(1*pd.get_dummies(readdata.copy()['vital_status'], drop_first=True)).ravel()

In [7]:
categorical_columns = [col for col in df.columns if df[col].dtype == "object"]
print(f"The categorical columns are: {categorical_columns} and the length is {len(categorical_columns)}.")

continuous_columns = [cols for cols in df.columns if df[cols].dtype in ["float64", "int64"]]
print(f"The continuous columns are: {continuous_columns} and the length is {len(continuous_columns)}.")

boolean_columns = [col_ for col_ in df.columns if df[col_].dtype == "bool"]
print(f"The boolean columns are: {boolean_columns} and the length is {len(boolean_columns)}.")

The categorical columns are: ['primary_diagnosis', 'prior_malignancy', 'prior_treatment', 'ethnicity', 'race'] and the length is 5.
The continuous columns are: ['ajcc_pathologic_stage', 'days_to_last_follow_up', 'year_of_diagnosis', 'pharm_treatment', 'radiation', 'age_at_index', 'year_of_birth'] and the length is 7.
The boolean columns are: [] and the length is 0.


In [8]:
# one hot encode categorical columns
df2 = df[categorical_columns].copy()
# Iterate through each column in the dataframe
for column in categorical_columns:
    one_hot = pd.get_dummies(df2[column], prefix=column).astype(int)
    # Drop the original column from the dataframe
    df2.drop(column, axis=1, inplace=True)
    # Join the new one-hot encoded columns to the original dataframe
    df2 = df2.join(one_hot)

In [9]:
# concatenate the continuous dataframe and one-hot encoded dataframe
new_df = pd.concat([df[continuous_columns], df2], axis=1)

new_df

,ajcc_pathologic_stage,days_to_last_follow_up,year_of_diagnosis,pharm_treatment,radiation,age_at_index,year_of_birth,"primary_diagnosis_Infiltrating duct carcinoma, NOS","primary_diagnosis_Lobular carcinoma, NOS",primary_diagnosis_others,...,prior_treatment_Not Reported,prior_treatment_Yes,ethnicity_hispanic or latino,ethnicity_not hispanic or latino,ethnicity_not reported,race_american indian or alaska native,race_asian,race_black or african american,race_not reported,race_white
0,1,337.0,2010.0,1,0,60,1950.0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
1,3,5.0,2010.0,0,0,56,1954.0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,759.0,2012.0,1,0,61,1951.0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
3,5,954.0,2010.0,1,1,71,1939.0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
4,8,304.0,2009.0,1,0,76,1933.0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,1,2813.0,2003.0,1,1,60,1943.0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
987,8,5.0,2013.0,0,0,35,1978.0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
988,1,1532.0,2009.0,1,0,46,1963.0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
989,3,375.0,2011.0,1,1,71,1940.0,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [11]:
# save the one hot encoded data

new_df.to_csv("./May 30/one_hot_encoded_data.csv", index=False)

In [18]:
random.seed(123)

# first split data into train and test
xtrain_, xtest, ytrain_, ytest = train_test_split(new_df, y, train_size=0.8, shuffle=True, stratify=y, random_state=123)

# scale data
scale = MinMaxScaler()
xtrain_ = scale.fit_transform(xtrain_)
xtest = scale.transform(xtest)

# perform stratified k-fold to split xtrain_ into xtrain_train and validation sets
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

basemodel = DecisionTreeClassifier(random_state=123)
best_model = None
best_val_score = -np.inf

base_scores = []
model_scores = []

# Perform stratified k-fold split with nested validation set
for fold, (train_idx, val_idx) in enumerate(skf.split(xtrain_, ytrain_)):
    xtrain_train, ytrain_train = xtrain_[train_idx], ytrain_[train_idx]
    xval, yval = xtrain_[val_idx], ytrain_[val_idx]

    # train base model
    basemodel.fit(xtrain_train, ytrain_train)
    base_val_pred = basemodel.predict(xval)
    base_val_score = accuracy_score(yval, base_val_pred)

    base_scores.append(base_val_score)

    # Train multiple models
    models = {'rf': RandomForestClassifier(random_state=123),
              'adaboost': AdaBoostClassifier(algorithm='SAMME'),
              'svc': SVC(),
              'logreg': LogisticRegression(max_iter=1000),
              'knn': KNeighborsClassifier(n_neighbors=10),
              'mlp': MLPClassifier(max_iter=10000)
              } # decisiontreeclassifier is our base model, thus we compare our results to this model.

    for model_name, model in models.items():
        # Train the model
        model.fit(xtrain_train, ytrain_train)
        # Validate the model
        val_predictions = model.predict(xval)
        val_score = accuracy_score(yval, val_predictions)

        #print(f"Fold {fold + 1}, Model: {model_name}, Validation Accuracy: {val_score:.4f}")

        model_scores.append((model_name, val_score))

In [19]:
# Print average accuracy of the base model
average_base_score = np.mean(base_scores)
print(f"Average Validation Accuracy of the Base Model: {average_base_score:.4f}")

# Calculate average validation accuracy for each model and find the best model
average_model_scores = {model_name: np.mean(scores) for model_name, scores in model_scores}
best_model_name = max(average_model_scores, key=average_model_scores.get)
best_model_avg_score = average_model_scores[best_model_name]

print(f"Best Model: {best_model_name} with Average Validation Accuracy: {best_model_avg_score:.4f}")

Average Validation Accuracy of the Base Model: 0.8649
Best Model: rf with Average Validation Accuracy: 0.8987


In [20]:
param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30]
    }
grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=skf, scoring='accuracy')

# Fit the grid search
grid_search.fit(xtrain_, ytrain_)
best_tuned_model = grid_search.best_estimator_
print(f"Best Model Hyperparameters: {grid_search.best_params_}")

# Test the best tuned model on the test set
test_predictions = best_tuned_model.predict(xtest)
test_score = accuracy_score(ytest, test_predictions)
print(f"Test Accuracy of the Best Tuned Model: {test_score:.4f}")

Best Model Hyperparameters: {'max_depth': 10, 'n_estimators': 100}
Test Accuracy of the Best Tuned Model: 0.9296
